<p style="font-family: 'Courier New', Courier, monospace; font-size: 30px; font-weight: bold; color: blue;  text-align: left;">
XGBoost for Regression 
</p>

In [2]:
# Libraries for data manipulation and visualization
import numpy as np                               # For numerical operations
import pandas as pd                              # For data manipulation
import matplotlib.pyplot as plt                  # For plotting
import seaborn as sns                            # For advanced data visualization

# Libraries for model building and evaluation
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (                     # For model evaluation metrics
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error, 
    median_absolute_error
)

# XGBoost library
import xgboost as xgb                             # For XGBoost Regressor
from xgboost import XGBRegressor

# ================================
# Set Seed for Reproducibility
# ================================
RANDOM_STATE = 50
np.random.seed(RANDOM_STATE)

In [3]:
# Define the path to the dataset
dataset_path = '../../all_data_files/cleaned_dataset_per_device.csv'

# Load the dataset
try:
    df = pd.read_csv(dataset_path)
    print("\nDataset loaded successfully...")
except FileNotFoundError:
    print(f"File not found at the specified path: {dataset_path}")
    import sys
    sys.exit()

# Display dataset information
print("\nDataset Information:")
df.info()
print("\nFirst Five Rows of the Dataset:")
display(df.head())


Dataset loaded successfully...

Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1415654 entries, 0 to 1415653
Data columns (total 20 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   time         1415654 non-null  object 
 1   device_id    1415654 non-null  object 
 2   co2          1415654 non-null  float64
 3   humidity     1415654 non-null  float64
 4   pm25         1415654 non-null  float64
 5   pressure     1415654 non-null  float64
 6   temperature  1415654 non-null  float64
 7   rssi         1415654 non-null  float64
 8   snr          1415654 non-null  float64
 9   SF           1415654 non-null  int64  
 10  frequency    1415654 non-null  float64
 11  f_count      1415654 non-null  float64
 12  p_count      1415654 non-null  float64
 13  toa          1415654 non-null  float64
 14  distance     1415654 non-null  int64  
 15  c_walls      1415654 non-null  int64  
 16  w_walls      1415654 non-null  int64

,time,device_id,co2,humidity,pm25,pressure,temperature,rssi,snr,SF,frequency,f_count,p_count,toa,distance,c_walls,w_walls,exp_pl,n_power,esp
0,2024-09-26 11:01:52.383162+00:00,ED3,645.0,54.18,0.32,300.48,23.87,-74.0,12.5,9,867.7,95.0,105.0,0.246784,18,1,2,91.26,-86.737602,-74.237602
1,2024-09-26 11:02:08.387851+00:00,ED0,539.0,49.34,0.39,299.69,24.57,-48.0,8.5,9,867.5,82.0,109.0,0.246784,10,0,0,65.26,-57.073822,-48.573822
2,2024-09-26 11:02:14.517123+00:00,ED1,471.0,45.29,0.32,300.80,25.64,-56.0,12.5,9,867.1,81.0,107.0,0.246784,8,1,0,73.26,-68.737602,-56.237602
3,2024-09-26 11:02:24.466271+00:00,ED2,659.0,49.09,0.44,301.18,24.57,-70.0,9.0,8,868.5,77.0,113.0,0.133632,23,0,2,87.26,-79.514969,-70.514969
4,2024-09-26 11:02:26.872032+00:00,ED4,423.0,38.86,0.13,301.69,27.71,-90.0,11.5,8,867.9,3597.0,4224.0,0.133632,37,0,5,107.26,-101.797062,-90.297062


In [4]:
# Define feature columns and target
feature_columns = [
    'distance', 'frequency', 'c_walls', 'w_walls', 
    'co2', 'humidity', 'pm25', 'pressure', 
    'temperature', 'snr'
]
target_column = 'exp_pl'

# Verify that all required columns exist
missing_columns = set(feature_columns + [target_column]) - set(df.columns)
if missing_columns:
    raise ValueError(f"The following required columns are missing in the dataset: {missing_columns}")

# Extract features and target
all_features = df[feature_columns].values
PL_all = df[target_column].values

# Perform train-test split (80-20 split)
X_train_all, X_test_all, PL_train_all, PL_test_all = train_test_split(
    all_features, PL_all, test_size=0.2, random_state=RANDOM_STATE
)

print("\nTrain-test split completed...\n")


Train-test split completed...



In [5]:
# Define a single set of hyperparameters manually
hyperparameters = {
    'max_depth': 10,
    'n_estimators': 300,
    'learning_rate': 0.05,
    'subsample': 0.9,
    'colsample_bytree': 0.01,      
    'colsample_bylevel': 0.01,      
    'colsample_bynode': 0.01, 
    'min_child_weight': 20,
    'gamma': 0.01,
    'reg_alpha': 1,
    'reg_lambda': 1,
    'objective': 'reg:squarederror',
    'random_state': RANDOM_STATE,
    'n_jobs': -1
}

print("\nTraining XGBoost model with hyperparameters.....\n")

# Initialize the XGBoost Regressor with the specified hyperparameters
xgb_model = xgb.XGBRegressor(**hyperparameters)

# Train the model
xgb_model.fit(X_train_all, PL_train_all)

print("Model training completed....!\n")


Training XGBoost model with hyperparameters.....

Model training completed....!



In [6]:
# Initialize a list to store evaluation metrics for the model
evaluation_metrics = []

print("\nEvaluating the trained XGBoost model...")

# Make predictions on the training set
PL_train_pred = xgb_model.predict(X_train_all)
# Make predictions on the test set
PL_test_pred = xgb_model.predict(X_test_all)

# Calculate metrics
train_mse = mean_squared_error(PL_train_all, PL_train_pred)
test_mse = mean_squared_error(PL_test_all, PL_test_pred)
train_r2 = r2_score(PL_train_all, PL_train_pred)
test_r2 = r2_score(PL_test_all, PL_test_pred)
test_rmse = np.sqrt(test_mse)
test_mape = mean_absolute_percentage_error(PL_test_all, PL_test_pred)
test_median_ae = median_absolute_error(PL_test_all, PL_test_pred)

# Append metrics to the list
evaluation_metrics.append({
    'Model': 'XGBoost',
    'Training Loss (MSE)': train_mse,
    'Test Loss (MSE)': test_mse,
    'Test RMSE': test_rmse,
    'R² Score': test_r2,
    'Test MAPE (%)': test_mape * 100,
    'Test Median AE': test_median_ae
})

# Convert the metrics list to a DataFrame
evaluation_df = pd.DataFrame(evaluation_metrics)

print(f"\nModel Evaluation Metrics:")
# Display the transposed DataFrame
evaluation_df = evaluation_df.T[1:].reset_index(names=["Metric"]).rename(columns={0: "Value"})
display(evaluation_df)


Evaluating the trained XGBoost model...

Model Evaluation Metrics:


,Metric,Value
0,Training Loss (MSE),47.687337
1,Test Loss (MSE),48.009054
2,Test RMSE,6.928857
3,R² Score,0.867691
4,Test MAPE (%),5.820572
5,Test Median AE,3.893633


In [7]:
# Define the number of folds for cross-validation
n_folds = 5

# Initialize KFold with consistent RANDOM_STATE
kf = KFold(n_splits=n_folds, shuffle=True, random_state=RANDOM_STATE)

# Initialize a list to store evaluation metrics for each fold
cv_results = []

print("\nPerforming Manual K-Fold Cross-Validation...\n")

# Iterate over each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_all), 1):

    # Split the data for the current fold
    X_train_fold, X_val_fold = X_train_all[train_idx], X_train_all[val_idx]
    PL_train_fold, PL_val_fold = PL_train_all[train_idx], PL_train_all[val_idx]
    
    # Initialize the XGBoost Regressor with the specified hyperparameters
    xgb_model = xgb.XGBRegressor(**hyperparameters)
    
    # Train the model on the current fold
    xgb_model.fit(X_train_fold, PL_train_fold)
    
    # Make predictions on the validation set
    PL_val_pred = xgb_model.predict(X_val_fold)
    
    # Calculate evaluation metrics
    val_mse = mean_squared_error(PL_val_fold, PL_val_pred)
    val_rmse = np.sqrt(val_mse)
    val_r2 = r2_score(PL_val_fold, PL_val_pred)
    val_mape = mean_absolute_percentage_error(PL_val_fold, PL_val_pred)
    val_median_ae = median_absolute_error(PL_val_fold, PL_val_pred)
    
    # Append metrics to the cv_results list
    cv_results.append({
        'Fold': fold,
        'Validation Loss (MSE)': round(val_mse, 4),
        'Validation RMSE': round(val_rmse, 4),
        'R² Score': round(val_r2, 4),
        'Validation MAPE (%)': round(val_mape * 100, 2),
        'Validation Median AE': round(val_median_ae, 4)
    })
    
    # Print metrics for the current fold
    print(f" Fold {fold} - MSE: {val_mse:.4f}, RMSE: {val_rmse:.4f}, R²: {val_r2:.4f}, "
          f"MAPE: {val_mape*100:.2f}%, Median AE: {val_median_ae:.4f}\n")

# Convert the list of dictionaries to a DataFrame for better visualization
cv_results_df = pd.DataFrame(cv_results)

print("K-Fold Cross-Validation Results:")
display(cv_results_df)

# Summary statistics
cv_summary = cv_results_df.agg(['mean', 'std']).round(4).reset_index()
cv_summary.rename(columns={'index': 'Metric'}, inplace=True)

# Transposing the summary
cv_summary_transposed = cv_summary.set_index('Metric').T

print("\nCross-Validation Summary:")
display(cv_summary_transposed)


Performing Manual K-Fold Cross-Validation...

 Fold 1 - MSE: 48.2265, RMSE: 6.9445, R²: 0.8680, MAPE: 5.84%, Median AE: 3.9020

 Fold 2 - MSE: 47.7398, RMSE: 6.9094, R²: 0.8694, MAPE: 5.82%, Median AE: 3.9053

 Fold 3 - MSE: 47.3289, RMSE: 6.8796, R²: 0.8698, MAPE: 5.80%, Median AE: 3.8809

 Fold 4 - MSE: 47.6434, RMSE: 6.9024, R²: 0.8693, MAPE: 5.81%, Median AE: 3.8824

 Fold 5 - MSE: 48.1104, RMSE: 6.9362, R²: 0.8678, MAPE: 5.83%, Median AE: 3.8998

K-Fold Cross-Validation Results:


,Fold,Validation Loss (MSE),Validation RMSE,R² Score,Validation MAPE (%),Validation Median AE
0,1,48.2265,6.9445,0.8680,5.84,3.9020
1,2,47.7398,6.9094,0.8694,5.82,3.9053
2,3,47.3289,6.8796,0.8698,5.80,3.8809
3,4,47.6434,6.9024,0.8693,5.81,3.8824
4,5,48.1104,6.9362,0.8678,5.83,3.8998



Cross-Validation Summary:


Metric,mean,std
Fold,3.0000,1.5811
Validation Loss (MSE),47.8098,0.3633
Validation RMSE,6.9144,0.0263
R² Score,0.8689,0.0009
Validation MAPE (%),5.8200,0.0158
Validation Median AE,3.8941,0.0115
